<img src="https://s3.amazonaws.com/southfloridareporter/wp-content/uploads/2018/08/02194523/reddit-1-logo-png-transparent.jpg" >

# Reddit APIs & Classification

## Content:
1. [Problem statement](#Problem-statement)
2. [Library importing](#*Library-importing*)
3. [Data scraping](#Data-scraping)
4. [Data cleaning](#Data-cleaning)
5. [Data Dictionary](#Data-Dictionary)
5. [Text preprocessing](#Text-preprocessing)
6. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
7. [Text analysis](#Text-analysis)
8. [Modeling](#Modeling)
9. [Best model evaluation](#Best-model-evaluation)
10. [Word inspection](#Word-inspection)
11. [Misclassification analisys](#Misclassification-analisys)
12. [Further research suggestions](#Further-research-suggestions)

## Problem statement

### Evaluation

Models are evaluated by **Classification accuracy, precision, recall and f1 score**.

[Accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) is the total percent of predictions a model gets right.

$$\text{Accuracy}={\frac{True Positive + True Negative}{True Positive + True Negative + False Positive + False Negative}}$$
_____
[Precision](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html) is the ratio of correctly predicted positive observations to the total predicted positive observations.

$$\text{Precision}={\frac{True Positive}{True Positive + False Positive}}$$
_____
[Recall](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html) is the ratio of correctly predicted positive observations to the all observations in positive class.

$$\text{Recall = Sensitivity}={\frac{True Positive}{True Positive + False Negative}}$$
_____
[F1](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) score is weighted average of the precision and recall. F1 score reaches its best value at 1 and worst score at 0.

$$\text{Recall = Sensitivity}={\frac{2 * Precision * Recall}{Precision + Recall}}$$


#### The goals of project:

- Using Reddit's API, collect posts from two subreddits (Psychology and Astrilogy).
- Use NLP to train a classifier on which subreddit a given post came from. This is a binary classification problem.

#### The resulting model can be useful for layman people who cannot decide if the information comes from psychological science or astrological pseudoscience.

#### *Library importing*

In [689]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
from datetime import datetime as dt
import re

import requests
from bs4 import BeautifulSoup


from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim.parsing import preprocessing
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,roc_auc_score

from wordcloud import WordCloud, STOPWORDS 
%%capture
from tqdm.notebook import tqdm
tqdm().pandas()
from IPython.display import display_html

UsageError: Line magic function `%%capture` not found.


## Data scraping

The data was scraped from [reddit](https://www.reddit.com/)

The finction collects text (title+selftext) (str), number of comments (int), number of ups (int), date of post (timestamp/float), subreddit names (str).

In [ ]:
def reddit_scraper(link_list,num_iter=10,act=True):
    """
    Scrapes text and subreddit from links on reddit.com (json)
    Saves info to csv to ./data/
    
    Takes:
        act=True - (bool) - activates the function.
        link_list - list of links to subreddits.
        num_iter=10 (int) - number of iteration through links (100 posts per iter).
    
    Returns:
        Reading from saved csv with:
            text (title+selftext) (str),
            number of comments (int),
            number of ups (int),
            date of post (timestamp/float),
            subreddit names (str)
    """
    # Creating empty list for posts
    if act:
        result = []
        posts = []
        # Loopong through links
        for link in tqdm(link_list):
            after = None
            for a in tqdm(range(num_iter)):
                params = {'after': after,'limit':200}
                res = requests.get(link,params =params,headers={'User-agent': 'Unicorn Tail'})
                if res.status_code != 200:
                    print('Status error', res.status_code)
                    break
                # Scrapes data from links and add info to posts list
                dict_json = res.json()
                posts.extend(dict_json['data']['children'])
                after = dict_json['data']['after']
                # generate a random sleep duration to look more 'natural'
                sleep_duration = random.randint(1,2)
                time.sleep(sleep_duration)
            print(f'Done for {link[25:-5]}')

        # Adding title + selftext, num_comments, ups, date and subreddit to result
        for row in range(len(posts)):
            text = posts[row]['data']['title']+' '+posts[row]['data']['selftext']
            result.append({'text':text,
                        'num_comments':posts[row]['data']['num_comments'],
                        'ups':posts[row]['data']['ups'],
                        'date':posts[row]['data']['created_utc'],
                        'subreddit':posts[row]['data']['subreddit'],})


        # Adding data to csv file
        pd.DataFrame(result).to_csv('./data/psy_astro.csv', index = False,header=True)
        # Returning DataFrame with info for saved file
        return pd.read_csv('./data/psy_astro.csv')

Scraping posts from r/psychology and r/astrology with 10 itteration (with limit = 100).

To activate scrapping change act=True

In [ ]:
%%time
df = reddit_scraper(['https://www.reddit.com/r/psychology.json',
                       'https://www.reddit.com/r/astrology.json',
                     ],num_iter=10,act=False)

## Data cleaning

In [ ]:
# Readding data from csv (to avoid overwrite data from scraping)
df = pd.read_csv('./data/psy_astro.csv')

In [ ]:
df.head()

We can see that scraper got texts, number of comments, number of ups, data and subreddit names correctly. Let's look at the shape of the data.

In [690]:
# Looking at the original shape of the dataset
df.shape

(1640, 7)

Some posts can be duplicated, we will drop all duplicates based on 'text' column.

In [691]:
# Dropping all duplicated rows
df.drop_duplicates(subset='text',inplace=True)

In [692]:
# Looking at the resulting shape of the dataset
df.shape

(1635, 7)

It can be seen that 230 rows were dropped because they had the same text. Now the dataset contains 1642 rows and 5 columns.

In [693]:
df['subreddit'].value_counts()

0    905
1    730
Name: subreddit, dtype: int64

In [694]:
# Looking at distribution of classes of subreddits
df['subreddit'].value_counts(normalize=True)

0    0.553517
1    0.446483
Name: subreddit, dtype: float64

The number of 'astrology' rows (907/0.55%) exceeds the number of 'psychology' rows (735/0.45%), the dataset is almost balanced, so we will not delete any other rows. 

In [695]:
# Checking for null values
df.isnull().sum()

text            0
num_comments    0
ups             0
date            0
subreddit       0
len_text        0
hours           0
dtype: int64

There is no null values. Let's check for data types of columns.

In [696]:
df.dtypes

text                    object
num_comments             int64
ups                      int64
date            datetime64[ns]
subreddit                int64
len_text                 int64
hours                    int64
dtype: object

All data types are good except date because the timestamp is unclear for understanding. The convertation of timestamps to date will be executed below.

In [697]:
# Convetring timestamp to date
df['date'] = df['date'].apply(lambda x: dt.fromtimestamp(x))
df['date'].head()

TypeError: an integer is required (got type Timestamp)

In [ ]:
df.dtypes

Now the dates are understandable.

The clean is saved to csv file clean_subreddits.csv

In [ ]:
df.to_csv('./data/clean_subreddits.csv')

## Data Dictionary

| Feature name | Type           | Description                                |
| ------------ | -------------- | ------------------------------------------ |
| text         | str            | Title + self text of post                  |
| num_comments | int            | Number of comments under a post            |
| ups          | int            | Number of ups of post                      |
| date         | datetime64[ns] | Date of publication of post                |
| subreddit    | str            | Name of subreddit (Psychology / Astrology) |



## Text preprocessing

Next step will be text preprocessing for the following exploratory data analysis.

In [ ]:
# removing tegs
df['text'] = df['text'].map(lambda x: BeautifulSoup(x).get_text())

In [ ]:
# removing non-letters
df['text'] = df['text'].map(lambda x:re.sub("[^a-zA-Z]", " ", x))

In [ ]:
# Instantiating Tokenizer and setting a pattern to only words
# applying Tokenizer to texts
tokenizer = RegexpTokenizer(r'\w+')
df['text'] = df['text'].map(lambda x: tokenizer.tokenize(x.lower()))

In [ ]:
# Checking for the result
df['text'][2][:10]

Now each documents look like list of words.

## Exploratory Data Analysis

Let's explore the dates when posts were published.

In [ ]:
# # Sorting table based on date
df.sort_values(by='date',ascending=False)

Posts in the dataset were published between 2020-05-13 21:05:19 and 2020-02-28 05:16:12. It means that the most recent data in 4 months were scraped.

Only one post was published 2019-04-11 04:19:34 - the text from the post points out that the post is a moderator's post, the row can be removed then.

In [ ]:
df.drop(index=962,inplace=True)

To compare text metrics, a new column with a length of texts will be created.

In [ ]:
# Creating new feature of length of texts by words
df['len_text'] = df['text'].apply(lambda x:len(x))

Let's see the length of documents in details.

In [ ]:
df.groupby('subreddit')['len_text'].sum()

Astrology subreddit's posts have significantly more words than Psychology. 

In [ ]:
df.groupby('subreddit')['len_text'].describe()

Mean and median of the number of words also very vary by subreddits. The average astrological document has 95 (mean) and 42.5 (median) words, although psychological documents have fewer words: 21 (mean) and 13 (median). The standard deviation for psychology is only 21 word and 177 for astrology. The minimum numbers of words in posts are 2 and 3, the maximum are significantly vary (almost 8 times): 2033 words for astrology and 274 for psychology.


It is noticeable that the mean and median differ, the standard deviation in astrological documents are high (177 words) which means that there are some outliers.

In [ ]:
df[df['len_text'] <= 3]

14 documents have 3 or fewer words in them. Most of them still can be recognized by specific words for a topic. Some of the short texts have many comments and ups. These posts could contain videos or photos which are not out of our project scope but could be explored further in other projects.

In [ ]:
df[df['len_text'] > 200]['subreddit'].value_counts()

Only 3 documents consist of more than 300 words in 'Psychology' and 91 in 'Astrology'.

In [ ]:
# Creating distribution of number of words in documents by subreddits
plt.figure(figsize=(10,6))

psy = df[df['subreddit']=='psychology']['len_text']
astr = df[df['subreddit']=='astrology']['len_text']

sns.distplot(psy,label='psychology',color='#65a8a7',bins=30)
sns.distplot(astr,label='astrology',color='#fcba03',bins=30)
plt.legend(loc='upper right',fontsize=15)
plt.title('Distribution of number of words in documents\n',fontsize=18)
plt.xticks(range(0,2150,150))
plt.show()

The distributions differ by subreddits. But both distributions are very skewed towards the right. That means few posts have a significantly larger number of words than average. But psychological  posts' length distributed steeper than astrological.

In [ ]:
# Creating a boxplot for text length by subreddits
df.boxplot(column='len_text',by='subreddit',figsize=(8,6),vert=False)
plt.title('Difference in subreddits\' text length\n\n')
plt.ylabel('Number of words');

It is clear that astrological posts have many outliers and some anomalies.

We can conclude that people who write posts for an astrological subreddit prefer to use more words (and sometimes even more than 1000 words) than those who writing posts for a psychological subreddit.

Number of comments will be explored next.

In [ ]:
# Sorting table based on number of comments
df.sort_values(by='num_comments',ascending=False)

It can be seen that there is one row (index 1866) that contains an anomaly number of comments - 1192. The row will be removed for better analysis of distribution but it will be kept separately for potential individual exploration.

In [ ]:
# Dropping a row with an anomaly and saving it separetely
most_commented_post = df.loc[1866,:].copy(deep=True)
df.drop(index=1866,inplace=True)
# Astrology post with an outstanding number of comments
most_commented_post['text']

In [ ]:
# Basic statistics of number of comments
df.groupby('subreddit')['num_comments'].describe()

Astrological community writing more comments to posts than psychological one: mean for astrology is 20 (8 median) and 12 (1 median) for psychology. Difference between median and mean indicates the presence of outliers. Visualisation of a number of commentaries by subreddits presented below.

In [ ]:
# Creating a boxplot and distribution for number of comments by subreddits
fig, ax = plt.subplots(nrows=1, ncols=2,squeeze=False,figsize=(15,5))
psy = df[df['subreddit']=='psychology']['num_comments']
astr = df[df['subreddit']=='astrology']['num_comments']

sns.boxplot(y='num_comments', x='subreddit', data=df, width=0.5,
                 palette="BrBG",ax=ax[0, 0])

sns.distplot(psy,label='psychology',color='#65a8a7',bins=30)
sns.distplot(astr,label='astrology',color='#fcba03',bins=30)
plt.suptitle('Difference in subreddits\' in number of comments',fontsize=16)
plt.legend(loc='upper right',fontsize=13)
plt.ylabel('Distribution')
plt.xlabel("Number of comments");

From visualization above (based on high of the boxplot and less skewness of distribution towards the right) we can conclude that Astrological community members are involved more in feedbacks under the posts than psychological community. Some astrological posts have outstanding number of comments.

A number of ups will be explored below.

In [ ]:
# Sorting table based on number of ups
df.sort_values(by='ups',ascending=False)

In [ ]:
# Creating a boxplot and distribution for number of ups by subreddits
fig, ax = plt.subplots(nrows=1, ncols=2,squeeze=False,figsize=(15,5))
psy = df[df['subreddit']=='psychology']['ups']
astr = df[df['subreddit']=='astrology']['ups']

sns.boxplot(y='ups', x='subreddit', data=df, width=0.5,
                 palette="BrBG",ax=ax[0, 0])

sns.distplot(psy,label='psychology',color='#65a8a7')
sns.distplot(astr,label='astrology',color='#fcba03')
plt.suptitle('Difference in subreddits\' in ups\n\n',fontsize=16)
plt.legend(loc='upper right',fontsize=13)
plt.ylabel('Distribution')
plt.ylabel('Number of ups');

In [ ]:
# Basic statistics of number of ups
df.groupby('subreddit')['ups'].describe()

The table above demonstrates that posts with the highest number of ups are all psychological. The boxplots and distributions illustrate the same trend: psychological community more than astrological one rate posts with ups. From this, we can conclude that feedbacks for posts in psychological subreddit carried out with ups although in astrological subreddit based on comments.

Next step is the exploration of the time pattern of posts published.

In [ ]:
# Extracting hours from date when posts were published
df['hours'] = df['date'].map(lambda x: x.hour)

In [ ]:
# Distribution of posts by the hour they post in by subreddits
plt.figure(figsize=(10,6))
psy = df[df['subreddit']=='psychology']['hours']
astr = df[df['subreddit']=='astrology']['hours']

sns.boxplot(y='ups', x='subreddit', data=df, width=0.5,
                 palette="BrBG",ax=ax[0, 0])

sns.distplot(psy,label='psychology',color='#65a8a7')
sns.distplot(astr,label='astrology',color='#fcba03')
plt.legend(loc='upper right',fontsize=13)
plt.xticks(range(24))
plt.ylabel('Distribution')
plt.ylabel('Number of ups');

There is a noticeable pattern of hours when posts were published. Majority of posts were published between 9 p.m and 3 a.m time, interestingly, that people prefer to publish their posts at night or very early morning. Much fewer people active in publication during lunch and after the lunchtime period (12-15 hours). Psychological posts were published later at night than astrological. Morning activity (from 8 to 10 a.m) can be seen for astrological subreddit.

In [ ]:
# =========== UNCOMMENT ===================
# Pairplot for all columns in dataset hue by subreddits
# sns.pairplot(df,hue='subreddit',palette="BrBG");

The number of comments plotted against the number of ups and number of ups by hours showed visual separation between two subreddits.

In [ ]:
# Matrix of intercorrelations
df.corr()

In [ ]:
# Visualisation of matrix of intercorrelations
sns.heatmap(df.corr(),annot=True,cmap="BrBG")
plt.title('Correlation of features\n');

From the matrix of intercorrelations above can be seen that only ups and number of comments have a moderate correlation (0.47 coefficient). Other columns do not correlate.

## Text analysis

In [ ]:
# Creating a function for t ext preprocessing
def text_preprocessing(df,columns_list,is_lem=True,is_stem=True):
    '''
    Lemmatize, Stemming list of words and concatenates in one string
    
    Takes:
        df - DataFrame
        columns_list - (list if str) - list with column' names with list of words
        is_lem=True - (bool) - activate WordNetLemmatizer
        is_stem=True - (bool) - activate PorterStemmer
    
    Returns:
        DataFrame with concatenated list of words
    '''
    lemmatizer = WordNetLemmatizer()
    p_stemmer = PorterStemmer()
    for column in columns_list:
        if is_lem:
            df[column] = df[column].apply(lambda row: [lemmatizer.lemmatize(text)
                                     for text in row])
        if is_stem:
            df[column] = df[column].apply(lambda row: [p_stemmer.stem(text)
                                     for text in row])
        df[column] = df[column].apply(lambda row: ' '.join(word for word in row))
    return df

In [ ]:
# Concatenating list of words in strings row-wise, 
# disabling Lemmatizer and PorterStemmer for better readability of words, 
# number of words in posts is not high so Lemmatizer and PorterStemmer are not necessary
df = text_preprocessing(df,['text'],is_lem=False,is_stem=False)

In [ ]:
df.head()

Converting the target variable from strings to ints. Psychology is a positive target.

In [ ]:
# Assigning 'psychology' as a positive class 1, astrology as 0
df['subreddit'] = df['subreddit'].map({'psychology':1,'astrology':0})

In [ ]:
# Assigning X to text and y to subreddits
X = df['text']
y = df['subreddit']

In [ ]:
# Dividing dataset into train and test subsets, saving the distribution ratio of the target
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.5,random_state=42)

In [ ]:
# Creating custom stopwords from two packages and words from subreddits 
# which clearly points out affiliation with subreddits to make classification more challenging
all_stop_words = set(list(stopwords.words('english')) + list(preprocessing.STOPWORDS) + 
                    ['psychology', 'astrology',
                     'psychological','astrological','psychologist','astrologist','https','www',
                    'reddit','com','s','th','r'])

In [ ]:
# 402 stopwords were defined
len(all_stop_words)

Let's visualize most frequent words in texts for both subreddits.

The code was taken from [here](https://www.geeksforgeeks.org/generating-word-cloud-python/) and modified.

In [ ]:
# Creating separated df for psychology and astrology
words = pd.DataFrame({'text':X_train,
                      'subreddit':y_train})
words_psy = words[words['subreddit']==1]
words_astr = words[words['subreddit']==0]

fig,ax = plt.subplots(1,2,figsize=(15,8))
stopwords = list(set(STOPWORDS))

# Converting separate documents into one string for each subreddit and creating wordclouds
comment_words_psy = ''
comment_words_psy += " ".join(word for word in words_psy['text'])+" "
wordcloud_psy = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = set(list(all_stop_words) + stopwords),
                min_font_size = 5).generate(comment_words_psy) 
  
comment_words_astr = ''
comment_words_astr += " ".join(word for word in words_astr['text'])+" "
wordcloud_astr = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = set(list(all_stop_words) + stopwords),
                min_font_size = 10).generate(comment_words_astr) 

# Plotting the WordCloud images
ax[0].imshow(wordcloud_psy)
ax[1].imshow(wordcloud_astr)
ax[0].set_title('Most frequent words in Psychological posts\n',fontsize=16)
ax[1].set_title('Most frequent words in Astrologival posts\n',fontsize=16)
plt.axis("off") 
plt.tight_layout(pad = 2) 
  
plt.show() 

It is clearly seen that only few words in both subreddits are repeating: people, new. Other words can be helpful for distinguishing subreddits. For psychology these words are study, research, anxiety, discussion, social etc. For Astrology are sign, chart, planet, house, mars, time ect.

Let's use CountVectorizer and TfidfVectorizer for comparison of most important words for classification.

In [ ]:
# Initializing two vectorizers
# maximum number of words is 5000, used custom stopwords
cvec = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = all_stop_words,
                             max_features = 5000)
tvec = TfidfVectorizer(stop_words = all_stop_words,
                             max_features = 5000)

In [698]:
# Vectorizing words for train dataset: fit the model and learn train vocabulary 
# and transforming strings info feature vectors

# CountVectorizer
train_features_cvec = cvec.fit_transform(X_train)
print(train_features_cvec.shape)

# TfidfVectorizer
train_features_tvec = tvec.fit_transform(X_train)
print(train_features_tvec.shape)

(820, 5721)
(820, 5721)


CountVectorizer to DataFrame

In [699]:
# Creating df from the model
df_words_cvec = pd.DataFrame(train_features_cvec.toarray(),
                  columns=cvec.get_feature_names())
df_words_cvec.head()

,aa,ab,abducted,abhor,abilities,ability,able,abnormal,abnormally,abound,...,zircan,zk,zodiac,zodiacal,zodiacs,zodiacum,zone,zoom,zwillinge,zxmoaplo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


TfidfVectorizer to DataFrame

In [700]:
# Creating df from the model
df_words_tvec = pd.DataFrame(train_features_tvec.toarray(),
                  columns=tvec.get_feature_names())
df_words_tvec.head()

,aa,ab,abducted,abhor,abilities,ability,able,abnormal,abnormally,abound,...,zircan,zk,zodiac,zodiacal,zodiacs,zodiacum,zone,zoom,zwillinge,zxmoaplo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.040109,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.401363,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Custom display of df side-by-side. 
CSS code was taken from [here](https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side)

In [701]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [702]:
# Displaying top 10 most frequent words from 
# TfidfVectorizer and CountVectorizer for both subreddits

top_10_words_cv = list(df_words_cvec.sum().sort_values(ascending=False).index)[:10]
df_words_cvec['is_psy'] = y_train.values
cv_train_small = df_words_cvec.groupby('is_psy')[top_10_words_cv].sum()

top_10_words_tv = list(df_words_tvec.sum().sort_values(ascending=False).index)[:10]
df_words_tvec['is_psy'] = y_train.values
tv_train_small = df_words_tvec.groupby('is_psy')[top_10_words_tv].sum()

display(cv_train_small.T)
display(tv_train_small.T)

# CountVectorizer           TfidfVectorizer

is_psy,0,1
moon,203,0
house,188,0
people,111,51
chart,161,1
like,149,11
venus,132,0
sign,129,0
sun,125,0
know,112,1
saturn,105,0


is_psy,0,1
moon,17.997949,0.000000
house,16.950563,0.000000
chart,15.707989,0.208593
people,7.443599,8.347398
sign,13.729905,0.000000
like,10.921245,1.608285
venus,11.493820,0.000000
know,10.495857,0.090052
new,4.398885,5.842805
study,0.810046,9.265572


Most important words for classification are similar, but CountVectorizer gives more common words such as th, know, like because it only counts the word frequencies. TfidfVectorizer adjusts its scores for the fact that some words appear more frequently in general hance gives more relevant words. The decision is to use TfidfVectorizer for our models.

## Modeling

Our baseline model's accuracy is 0.55 (55%) of predicting true subreddit for a post.

In [703]:
y_test.value_counts(normalize=True)

0    0.552439
1    0.447561
Name: subreddit, dtype: float64

Next step is fitting 5 different models.

3 Classifiers:
    - Naive Bayes: MultinomialNB because all columns are integer counts.
    - LogisticRegression 
    - KNeighborsClassifier 
And 2 Vectorizers:
    - CountVectorizer (GridSearch through parameters)
    - TfidfVectorizer (GridSearch through parameters)
    
Models will be evaluated based on 4 [metrics](#Evaluation): accuracy, precision, recall, f1.

In [704]:

def model_metrics(y_test, y_predicted):  
    """
    Calculates accuracy, precision, recall, f1
    
    Takes:
    y_test - pandas Series
    y_predicted - pandas Series
    
    Prints accuracy, precision, recall, f1
    
    Returns:
    None
    """
    # set of predicted labels match the corresponding set of true labels
    accuracy = accuracy_score(y_test, y_predicted)
    # ratio tp / (tp + fp)
    precision = precision_score(y_test, y_predicted)             
    # ratio tp / (tp + fn)
    recall = recall_score(y_test, y_predicted)
    # weighted average of the precision and recall
    # F1 = 2 * (precision * recall) / (precision + recall)
    f1 = f1_score(y_test, y_predicted)
    
    print(f'accuracy {round(accuracy,3)}, precision {round(precision,3)},recall {round(recall,3)}, f1 {round(f1,3)}')

In [705]:
def model_choosing(vectorizers,model,params,
                   X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test):
    '''
    Evaluate model with two vectorizers
    Takes:
    vectorizers - list of vectorizers
    model - initialized model name
    params - parameters for GridSearchCV
    X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test
    
    Prints  Cross_val_scores, 
            Best score of grid search,
            Best parameters,
            Test score
    
    Returns:
    None
    '''
    for vectorizer in vectorizers:
        print('============================')
        print(str(vectorizer).split('(')[0])
        print(str(model).split('(')[0])
        pipe = Pipeline([
                ('vect',vectorizer),
                ('model',model)
            ])
        pipe_params = params
        gs = GridSearchCV(pipe, 
                  param_grid=pipe_params,
                  cv=3) 
        cross_scores = cross_val_score(gs,X_train,y_train)
        gs.fit(X_train,y_train)
        print(f'Cross_val_scores: {cross_scores}')
        print(f'Best score of grid search: {gs.best_score_}')
        print(f'Best parameters: {gs.best_params_}')
        gs_model = gs.best_estimator_
        gs_model.fit(X_train,y_train)
        test_score = gs_model.score(X_test, y_test)
        print(f'Test score: {test_score}')

In [706]:
# Modeling LogisticRegression with CountVectorizer and TfidfVectorizer
# applying different hyperparameters

%%time
cvec = CountVectorizer(stop_words = all_stop_words)
tvec = TfidfVectorizer(stop_words = all_stop_words)
lr = LogisticRegression()
params = {           
            'vect__max_features':[2000,3000,5000],
            'vect__min_df':[1,2,3],
            'vect__max_df':[0.9,0.95],
            'vect__ngram_range': [(1,1), (1,2)]
            }
best_param = model_choosing([cvec,tvec],lr,params)

# Evaluating model with best hyperparameters
# printing cross_val_scores, train score, test score, accuracy, precision, recall, f1 score

print('====== Final model with best params for Vectorizer ======')
tvec = TfidfVectorizer(best_param,stop_words = all_stop_words)
X_train_tv = tvec.fit_transform(X_train)
X_test_tv = tvec.transform(X_test)
cross_scores = cross_val_score(lr,X_train_tv,y_train)
print(f'Cross_val_scores: {[round(i,3) for i in cross_scores]}')
lr.fit(X_train_tv,y_train)
train_score = lr.score(X_train_tv, y_train)
print(f'Train score: {round(train_score,3)}')
test_score = lr.score(X_test_tv, y_test)
print(f'Test score: {round(test_score,3)}')
predictions = lr.predict(X_test_tv)
model_metrics(y_test,predictions)

UsageError: Line magic function `%%time` not found.


In [ ]:
# Modeling KNeighborsClassifier with CountVectorizer and TfidfVectorizer
# applying different hyperparameters

%%time
knn_model = KNeighborsClassifier(n_neighbors=10)
params = {           
            'vect__max_features':[2000,3000,5000],
            'vect__min_df':[1,2,3],
            'vect__max_df':[0.9,0.95],
            'vect__ngram_range': [(1,1), (1,2)],
            }

model_choosing([cvec,tvec],knn_model,params)

# Evaluating model with best hyperparameters
# printing cross_val_scores, train score, test score, accuracy, precision, recall, f1 score

print('====== Final model with best params for Vectorizer ======')
tvec_knn = TfidfVectorizer(best_param,stop_words = all_stop_words)
X_train_tv_knn = tvec_knn.fit_transform(X_train)
X_test_tv_knn = tvec_knn.transform(X_test)
cross_scores = cross_val_score(knn_model,X_train_tv_knn,y_train)
print(f'Cross_val_scores: {[round(i,3) for i in cross_scores]}')
knn_model.fit(X_train_tv_knn,y_train)
train_score = knn_model.score(X_train_tv_knn, y_train)
print(f'Train score: {round(train_score,3)}')
test_score = knn_model.score(X_test_tv_knn, y_test)
print(f'Test score: {round(test_score,3)}')
predictions = knn_model.predict(X_test_tv_knn)
model_metrics(y_test,predictions)

In [ ]:
# Modeling MultinomialNB with TfidfVectorizer
# printing cross_val_scores, train score, test score, accuracy, precision, recall, f1 score

%%time
tvec = TfidfVectorizer(stop_words = all_stop_words)
nb = MultinomialNB()
X_train_tv = tvec.fit_transform(X_train)
X_test_tv = tvec.transform(X_test)
cross_scores = cross_val_score(nb,X_train_tv.todense(),y_train)
print(f'Cross_val_scores: {[round(i,3) for i in cross_scores]}')
nb.fit(X_train_tv.todense(),y_train)
train_score = nb.score(X_train_tv.todense(), y_train)
print(f'Train score: {round(train_score,3)}')
test_score = nb.score(X_test_tv.todense(), y_test)
print(f'Test score: {round(test_score,3)}')
predictions = nb.predict(X_test_tv)
model_metrics(y_test,predictions)

| Vectorizer      | Model                | Cross_val_scores_train (accuracy) | Train accuracy | Test accuracy | Test precision | Test recall | Test f1 |
| --------------- | -------------------- | --------------------------------- | -------------- | ------------- | -------------- | ----------- | ------- |
| TfidfVectorizer | MultinomialNB        | 0.902, 0.927, 0.915, 0.909, 0.884 | 0.993          | 0.921         | 0.984          | 0.837       | 0.904   |
| TfidfVectorizer | LogisticRegression   | 0.97, 0.933, 0.945, 0.957, 0.896  | 0.998          | 0.945         | 0.924          | 0.956       | 0.94    |
| TfidfVectorizer | KNeighborsClassifier | 0.866, 0.89, 0.909, 0.884, 0.878  | 0.913          | 0.904         | 0.965          | 0.815       | 0.883   |

As it is shown on the table above - LogisticRegression is the best model with TfidfVectorizer. We can see that cross-validation scores vary as well as train and test score. It means that our model is overfitted, but the test score is still very high and higher than other models. Also, LogisticRegression showed the best recall that means that classifier found almost all positive samples (psychological subreddit posts).

All the models perform better with TfidfVectorizer. MultinomialNB and KNeighborsClassifier also showed pretty decent scores. KNeighborsClassifier turned out to be less overfitted in comparison with others.

### Best model evaluation

In [ ]:
# Creating a dataframe with list of true values and predicted probabilities based on our model
pred_proba = [i[1] for i in lr.predict_proba(X_test_tv)]
pred_df = pd.DataFrame({'true_values': y_test,
                        'pred_probs':pred_proba})
pred_df.head()

In [ ]:
# Creating distribution of divided probability for two subreddits

plt.figure(figsize = (10,7))
# Create two histograms of observations.
plt.hist(pred_df[pred_df['true_values'] == 0]['pred_probs'],
         bins=25,
         color='#65a8a7',
         alpha = 0.5,
         label='Astrology')
plt.hist(pred_df[pred_df['true_values'] == 1]['pred_probs'],
         bins=25,
         color='#fcba03',
         alpha = 0.5,
         label='Psychology')

# Add vertical line at P(Outcome = 1) = 0.5.
plt.vlines(x=0.5,
           ymin = 0,
           ymax = 65,
           color='r',
           linestyle = '--')

# Label axes.
plt.title('Distribution of Probability', fontsize=22)
plt.ylabel('Frequency', fontsize=18)
plt.xlabel('Predicted Probability that Outcome = 1', fontsize=18)
plt.text(y = 40,x = 0.1,s = 'True negative',color='blue')
plt.text(y = 10,x = 0.25,s = 'False negative',color='orange')
plt.text(y = 10,x = 0.55,s = 'False positive',color='blue')
plt.text(y = 40,x = 0.75,s = 'True positive',color='orange')
# Create legend.
plt.legend(fontsize=20);

It can be seen that probabilities of Astrological and Psychological subreddits are differently distributed. The overlapped area is pretty small. It means that our best model classifying well.

In [ ]:
# Creating Receiver Operating Characteristic (ROC) Curve

plt.figure(figsize = (10,7))

# Create threshold values. (Dashed red line in image.)
thresholds = np.linspace(0, 1, 200)

# Define function to calculate sensitivity. (True positive rate.)
def TPR(df, true_col, pred_prob_col, threshold):
    true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
    false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
    return true_positive / (true_positive + false_negative)
    

# Define function to calculate 1 - specificity. (False positive rate.)
def FPR(df, true_col, pred_prob_col, threshold):
    true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
    false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
    return 1 - (true_negative / (true_negative + false_positive))
    
# Calculate sensitivity & 1-specificity for each threshold between 0 and 1.
tpr_values = [TPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]
fpr_values = [FPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]

# Plot ROC curve.
plt.plot(fpr_values, # False Positive Rate on X-axis
         tpr_values, # True Positive Rate on Y-axis
         label='ROC Curve')

# Plot baseline. (Perfect overlap between the two populations.)
plt.plot(np.linspace(0, 1, 200),
         np.linspace(0, 1, 200),
         label='baseline',
         linestyle='--')

# Label axes.
plt.title(f'ROC Curve with AUC = {round(roc_auc_score(pred_df["true_values"], pred_df["pred_probs"]),3)}', fontsize=22)
plt.ylabel('Recall', fontsize=18)
plt.xlabel('1 - Specificity', fontsize=18)

# Create legend.
plt.legend(fontsize=16);

ROC is a probability curve and AUC represents degree or measure of separability. It tells that our model is capable of distinguishing between two classes. AUC = 0.991, that means that positive and negative classes are separated good.

### Word inspection

Let's look at the coefficient of the words the prediction was made on.

In [ ]:
# Creating df with words, number of words, coefficients and exponent of coefficients
# We have to exponent out coefficient because LogisticRegression usr logit for calculation
predictors = pd.DataFrame({
    'word':list(tvec.get_feature_names()),
    'word_count':list(tvec.vocabulary_.values()),
    'coef':lr.coef_[0],
    'exp_coef':np.exp(lr.coef_)[0]
})
predictors.sort_values(by='coef',ascending=False).head(15)

The table above demonstrates words which have the highest coefficient in the prediction of psychological class. Some words are clearly from psychological vocabulary: psychreg (website name), brain, mental, anxiety, cognitive, behavior. But some words are from general vocabulary: finds, people, women, covid. The model can be better generalize for distinguishing psychological posts from other posts if these words will be revisited and may be removed.

In [ ]:
predictors.sort_values(by='coef',ascending=True).head(15)

The table above demonstrates words which have a negative coefficient in the prediction of psychological class. These words refer to astrological category. The words are clearly from astrological vocabulary: chart, moon, house, sign, saturn. But some words are general: know, like, think. For better performance of model, it would be better to add them to stopwords.

### Misclassification analisys

Let's look at mislassified posts.

In [ ]:
# Making predictions
predictions = lr.predict(X_test_tv)

In [ ]:
# Creating confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()

In [ ]:
print("True Astrology [True Negatives]: %s" % tn)
print("False Psychology [False Positives]: %s" % fp)
print("False Astrology [False Negatives]: %s" % fn)
print("True Psychology [True Positives]: %s" % tp)

In [ ]:
plot_confusion_matrix(lr,X_test_tv,y_test,normalize='true');

From the confusion matrix, it can be noticed that percent of misclassification is 6.4% for false positive (Astrological posts were classified as psychological) and 4.4% for false negative (Psychological posts were classified as astrological). True negative (astrological class) were identified a bit better than true positive (psychological class). But slightly unbalanced classes of data should be taken into account.

In [ ]:
# Looking at mislaccified posts
probabilities = pd.DataFrame(lr.predict_proba(X_test_tv),columns=['astr_proba','psy_proba'])
probabilities['subred'] = y_test.values
probabilities['predict'] = predictions
probabilities['text'] = X_test.values
probabilities['num_comments'] = list(df.loc[y_test.index,'num_comments'])
probabilities['ups'] = list(df.loc[y_test.index,'ups'])
probabilities['len_text'] = list(df.loc[y_test.index,'len_text'])
misclassified = probabilities[probabilities['subred']!=probabilities['predict']]

In [ ]:
display(misclassified[misclassified['subred'] ==1].head(34))
display(misclassified[misclassified['subred'] ==1].head(34))

Some posts which were classified as astrological have general information in them but others are clearly contained psychological vocabulary. These posts are quite different in terms of length of text (but mostly small number)and meaning of posts. Most probabilities fro psychology is higher than 0.5 so the possible solution will be to drop a threshold for classification.

Let's see false positive than.

In [ ]:
display(misclassified[misclassified['subred'] ==0].head(34))

All texts contain astrological vocabulary, the misclassification can be due to misspelling some words. Words such as astrology, astrological were removed what leads to the change of meaning of the texts. Also, some probabilities are high enough to separate the classes so changing a threshold can be a solution.

In [ ]:
# Word cloud of misclassified posts

comment_words = ''
comment_words += " ".join(word for word in misclassified['text'])+" "
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = all_stop_words, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.title('Words from misclassified posts')
plt.tight_layout(pad = 0)
  
plt.show() 

It is noticeable that most of the words are from general vocabulary.

## Conclusion

## Further research suggestions

Based on the model limitation some suggestions for further research can be made:

- Scrape more data for each subreddit.
- Use fully balanced classes for better predictions.
- Use the same balance of classes as a number of all posts in subreddits for better generalization of a model.
- Use additional features such as number of comments, number of ups and length of text for extension of the model.
- Make a decision about general words: some general words can lead to misclassification. 
- Make a decision about keeping specific words. Words such as psychology and astrology were removed from task complications and educational purpose only in this project.
- Filter posts with a small number of words in selftext and title.
- Use more advanced models.

Ideas for additional research:

- Exploring posts with a high number of commentaries and ups.
- Expend a model for more broad classification:
    - Distinguishing psychological posts from other posts with different topics.
    - Explore how psychology differs from science subreddit.
    - Add more than two classes.
- Build a model for posts with a small length of texts.
- Extend dataset with posts which were published during longer period of time.